In [274]:
import pandas as pd
from datetime import datetime

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.utils.np_utils import to_categorical

import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.externals import joblib

from sklearn import tree

In [275]:
train_dataframe = pd.read_csv("../data/chemsdata.csv", sep=',', index_col=0)
train_y = (np.array(train_dataframe.iloc[:,-1]) + 1)/2
train_matrix = pd.DataFrame.as_matrix(train_dataframe.iloc[:,:-1])
#train_matrix = np.delete(train_matrix, list(range(300,450)), axis = 0) # 643
#train_y = np.delete(train_y, list(range(300,450)), axis = 0)
#train_matrix = normalize(train_matrix, axis=0, norm='l2')

In [330]:
shuffle_permutation = np.random.permutation(len(train_y))

train_y = train_y[shuffle_permutation]
train_matrix = train_matrix[shuffle_permutation]

num_train = int(np.floor((len(train_matrix)*.8)))
num_validation = int(np.floor((len(train_matrix)*.2)))

In [331]:
dt = tree.DecisionTreeClassifier()
dt.fit(train_matrix[0:num_train], train_y[0:num_train])
print(dt.score(train_matrix[0:num_train], train_y[0:num_train]))
print(dt.score(train_matrix[-num_validation:], train_y[-num_validation:]))

1.0
0.8199052132701422


In [332]:
scaler = StandardScaler()
scaler.fit(train_matrix[num_train:])
train_scaled = scaler.transform(train_matrix)

pca = PCA()
pca = pca.fit(train_scaled[num_train:])
train_pca = pca.transform(train_scaled)

In [334]:
lda = LDA()
lda = lda.fit(train_pca, train_y)
print(lda.score(train_pca[0:num_train], train_y[0:num_train]))
print(lda.score(train_pca[-num_validation:], train_y[-num_validation:]))

0.8696682464454977
0.8909952606635071


In [371]:
model = Sequential()

model.add(Dense(3, activation='relu', input_shape = (train_pca.shape[1],)))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_127 (Dense)            (None, 3)                 126       
_________________________________________________________________
dense_128 (Dense)            (None, 6)                 24        
_________________________________________________________________
dense_129 (Dense)            (None, 3)                 21        
_________________________________________________________________
dense_130 (Dense)            (None, 1)                 4         
Total params: 175
Trainable params: 175
Non-trainable params: 0
_________________________________________________________________


In [374]:
model.fit(train_pca, train_y, epochs=250, batch_size=80, validation_split=.2)
best_val = model.evaluate(train_pca[-num_validation:], train_y[-num_validation:])
best_val

Train on 844 samples, validate on 211 samples
Epoch 1/250
844/844 [==============================] - 0s 33us/step - loss: 0.1807 - acc: 0.9254 - val_loss: 0.4250 - val_acc: 0.9005
Epoch 2/250
844/844 [==============================] - 0s 30us/step - loss: 0.1811 - acc: 0.9254 - val_loss: 0.4239 - val_acc: 0.9005
Epoch 3/250
844/844 [==============================] - 0s 32us/step - loss: 0.1808 - acc: 0.9254 - val_loss: 0.4253 - val_acc: 0.9005
Epoch 4/250
844/844 [==============================] - 0s 35us/step - loss: 0.1811 - acc: 0.9242 - val_loss: 0.4244 - val_acc: 0.9052
Epoch 5/250
844/844 [==============================] - 0s 31us/step - loss: 0.1806 - acc: 0.9254 - val_loss: 0.4253 - val_acc: 0.9052
Epoch 6/250
844/844 [==============================] - 0s 31us/step - loss: 0.1808 - acc: 0.9254 - 

[0.45250630576463674, 0.9052132707071531]

In [49]:
for iteration in range(5):
    model_copy = keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())
    model.fit(train_pca, train_y, epochs=20, batch_size=1000, validation_split=.25)
    val_score = model.evaluate(train_pca[-int(np.ceil((len(train_matrix)*.2))):], train_y[-int(np.ceil((len(train_matrix)*.2))):])[1]
    if val_score < best_val:
        model = model_copy
        model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    else:
        best_val = val_score

Train on 791 samples, validate on 264 samples
Epoch 1/20
791/791 [==============================] - 0s 19us/step - loss: 0.1519 - acc: 0.9558 - val_loss: 0.5183 - val_acc: 0.8977
Epoch 2/20
791/791 [==============================] - 0s 19us/step - loss: 0.1402 - acc: 0.9532 - val_loss: 0.5204 - val_acc: 0.8977
Epoch 3/20
791/791 [==============================] - 0s 24us/step - loss: 0.1570 - acc: 0.9545 - val_loss: 0.5212 - val_acc: 0.8977
Epoch 4/20
791/791 [==============================] - 0s 27us/step - loss: 0.1474 - acc: 0.9583 - val_loss: 0.5221 - val_acc: 0.9015
Epoch 5/20
791/791 [==============================] - 0s 28us/step - loss: 0.1500 - acc: 0.9494 - val_loss: 0.5230 - val_acc: 0.9015
Epoch 6/20
791/791 [==============================] - 0s 27us/step - loss: 0.1426 - acc: 0.9558 - val_loss: 0.5228 - val_acc: 0.9015
Epoch 7/20
791/791 [==============================] - 0s 24us/step - loss: 0.1491 - acc: 0.9621 - val_loss: 0.5232 - val_acc: 0.9015
Epoch 8/20
791/791 [===

In [ ]:
model.fit(train_pca, to_categorical(train_y, num_classes = 2), epochs=1200, batch_size=80, validation_split=.2)

In [68]:
model.evaluate(train_pca[-num_validation:], train_y[-num_validation:])

211/211 [==============================] - 0s 43us/step


[0.2846901022709941, 0.8862559227581838]

In [31]:
model.evaluate(new_train, new_train_y)

NameError: name 'new_train' is not defined

In [67]:
model.evaluate(train_pca[-int(np.ceil((len(train_matrix)*.2))):], to_categorical(train_y[-int(np.ceil((len(train_matrix)*.2))):], num_classes = 2))

ValueError: Error when checking target: expected dense_31 to have shape (1,) but got array with shape (2,)

In [123]:
model.evaluate(train_pca, to_categorical(train_y, num_classes = 2))

1055/1055 [==============================] - 0s 38us/step


[1.7161996170242815, 0.8834123214839195]

In [13]:
int(np.ceil((len(train_matrix)*.2)))

211

In [82]:
test_dataframe = pd.read_csv("../data/chemstest.csv", sep=',')
test_matrix = pd.DataFrame.as_matrix(test_dataframe.iloc[:,:-1])
test_pca = pca.transform(test_matrix)

In [87]:
test_predictions = model.predict_classes(test_pca)

In [88]:
for num in test_predictions[:,0]:
    print((num*2) - 1)

-1
-1
-1
1
1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
1
1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
-1
1
1
-1
-1
-1
1
1
-1
-1
-1
1
-1
-1
1
-1
1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
1
-1
1
-1
1
-1
1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
1
-1
-1
-1
1
-1
-1
1
1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
1
1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
1
-1
-1
1
-1
1
-1
-1
-1
-1
1
-1
1
1
-1
1
1
-1
-1
-1
-1
1
-1
-1
-1
1
1
1
-1
-1
-1
-1
1
-1
-1
1
-1
-1
1
-1
1
-1
-1
-1
-1
-1
1
-1
-1
1
-1
-1
1
-1
-1
-1
-1
-1
1
-1
1
-1
-1
-1
1
-1
1
-1
-1
1
-1
1
-1
-1
-1
1
1
1
1
-1
-1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
1
-1
1
-1
1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
1
-1
1
-